## Goals for this section
1. Use Julia high-performance IO packages to store the results of our simulations
2. Learn more about Julia's parallel computing capabilities with respect to partitioning work, storing pre-computed results, etc.
3. Branch our into the wild, wild world of condensed matter physics

Want a way to save/load our data.
We can use [HDF5](https://github.com/JuliaIO/HDF5.jl) for maximum portability or [JLD](https://github.com/JuliaIO/JLD.jl) if we are comfortable staying in the Julia ecosystem. Or, attend Simon's talk about [JLD2](https://github.com/simonster/JLD2.jl) for more information about cutting-edge IO in Julia! For now, we will stick with HDF5 for portability.

Some patterns we could use:

1. Have every worker send their data to the driver, and have it write one file
2. Have each worker write its own data file, then collate them at the end using some script
3. Have a group of workers dedicated to collecting data and collating

3 is intermediate between 1 and 2. Each one has advantages and disadvantages. 3 will let us show off some cool Julia features like `WorkerPool` and it's also the most difficult to get right, so let's do that!

In [ ]:
addprocs(6)

IOPool = WorkerPool([2; 3]) # worker "1" is the driver worker
ComputePool = CachingPool([4; 5; 6; 7])

We make `ComputePool` a [`CachingPool`](https://docs.julialang.org/en/latest/stdlib/parallel/#Base.Distributed.CachingPool) so that it can store information about the Hamiltonian between timesteps. Hopefully this will cut down on our runtimes! Now we want to:

1) Have `ComputePool` work on our time evolution function from last time (we'll modify our call to `pmap` to do this).

In [ ]:
function timeSeriesParallel(psi, H, start, step, stop; worker_pool::WorkerPool=default_worker_pool())
    times = linspace(start, step, stop)
    pmap(worker_pool, (t,)->timeEvolve(psi, H, t), times)
end

addprocs(6)
psi, H = get_groundstate(10, 1.0)
t_vecs = timeSeriesParallel(psi, H, 0.0, 0.1, 10.0, worker_pool = ComputePool)

2) For each eigenvector we get back from `ComputePool`, feed it to `IOPool` to write to disk

In [ ]:
@everywhere function WriteToDisk(vector, index, file_name_root)
    
end

3) Have the driver node make a plot for us in the meantime (we already have code to do this!)

And now we are ready to upload our data to Git LFS and head out to arXiv!

## Going Forward

Hopefully everyone got to learn some physics and some Julia (or at least how you can use Julia to do physics!). Some suggestions of next steps to try:
- any of the exercises you didn't manage to get through
- add disorder - make the $\Delta$ in the $XXZ$ model or $h$ in the transverse field Ising model vary by site $i$. Note that you'll probably need to *disorder average* - generate many (200+) Hamiltonians with different randomness samples and average over their results
- use your quench code to visualize [spin echo](https://en.wikipedia.org/wiki/Spin_echo) at the edge of the chain